In [1]:
import numpy as np
import pandas as pd
from skimage import feature
from skimage.io import imread
from skimage.io import imsave
from skimage.transform import integral_image
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklaern.feature_selection import RFECV

In [2]:
images = []
for i in range(1,72):
    img_name = str(i) + '.jpg'
    img = imread(img_name, as_gray = True)
    images.append(img)

In [3]:
lbp_code_1to71 = np.zeros((100), np.float64)

for i in range(100):
    if i < 71:
        int_img = integral_image(images[i])
        lbp_code = feature.multiblock_lbp(int_img, 0, 0, 3, 3)
        lbp_code_1to71[i] = lbp_code
    else:
        lbp_code_1to71[i] = np.nan

In [4]:
lbp_code_1to71

array([ 30.,  55., 248.,  12., 255., 241., 255.,  41.,   3.,   8.,  96.,
       223., 143., 248., 255.,  52.,  80., 120.,  12., 167., 224., 199.,
       225., 193., 207., 124., 143.,  60., 255.,  15., 124.,   0.,  16.,
         8., 131.,   0.,   0., 216.,  31., 120.,  64., 112., 192., 193.,
       120., 227., 124., 248., 192., 242., 240., 253.,  31., 168., 248.,
       151.,   8., 193.,  34.,  30., 254.,   0., 143.,  32., 225.,  16.,
       250.,  96.,  62.,  19., 241.,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan])

In [5]:
base = pd.read_csv('base_transformada_total.csv')

In [6]:
base.drop('Unnamed: 0',inplace = True, axis = 1)

In [7]:
base['MB-LBP'] = lbp_code_1to71

In [8]:
base_train = base.dropna()
base_toPredict = base[base['MB-LBP'].isnull()].drop('MB-LBP', axis = 1)

In [9]:
print(base_train.shape)
print(base_toPredict.shape)

(71, 32)
(29, 31)


In [11]:
randomForest = RandomForestRegressor(n_estimators=100)

In [12]:
randomForest.fit(X = base_train.drop('MB-LBP',axis=1), y = base_train['MB-LBP'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [13]:
y_pred = randomForest.predict(base_toPredict)

In [17]:
base['MB-LBP'].fillna(value = pd.Series(data = y_pred, index = base_toPredict.index), inplace = True)

In [18]:
base.isnull().sum()

Matriz mesangial                   0
Celularidade mesangial             0
Membrana basal glomerular          0
Espaço de Bowman                   0
Tufo glomerular                    0
Podócito                           0
Necrose tubular aguda              0
Vacuolização do epitélio           0
Cristais                           0
Calcificação                       0
Tubulite                           0
Cilindros                          0
Atrofia tubular                    0
Graduação de atrofia               0
Fibrose intersitical               0
Graduação da fibrose               0
Infiltrado inflamatório            0
tipo de infiltrado inflamatório    0
Vasculite                          0
Microangiopatia trombótica         0
Trombos                            0
Hialinose                          0
Necrose Fibrinóide                 0
IMUNOFLUORESCÊNCIA                 0
padrão de deposição                0
Imunodepósitos                     0
Imunohistoquímica                  0
M

In [ ]:
base.to_csv('base_MB-LBP.csv')